In [1]:
#pip install aiohttp asyncssh

In [3]:
import asyncio
import asyncssh
import aiohttp
from http.client import HTTPConnection
from urllib.parse import urlparse


async def site_is_online_async(url, timeout):
    """Return True if the target URL is online.

    Raise an exception otherwise.
    """
    error = Exception("unknown error")
    
    if urlparse(url).scheme in ("http", "https"):
        async with aiohttp.ClientSession() as session:
            try:
                await session.head(url, timeout=timeout,ssl=False)
                return True
            except asyncio.TimeoutError:
                error = Exception("timed out")
            except (asyncssh.Error) as e:
                error = e
                
    if urlparse(url).scheme in ("ssh"):
        parser = urlparse(url)
        host = parser.netloc or parser.path.split("/")[0]
        if  parser.port == None : 
            port=22 
        else: 
            port=parser.port
            host=host.split(":")[0]

        #print(host,port)
        
        try:
            async with await asyncio.wait_for(asyncssh.connect(host=host,port=port,username="test", password="test", known_hosts=None), timeout=timeout) as conn:
                print((await conn.run('tty', term_type='ansi')).stdout, end='')
                return True
        except asyncio.TimeoutError:
                error = Exception("timed out")
        except (asyncssh.Error) as e:
                error = e
            
    raise error
    
def display_check_result(result, url, error=""):
    """Display the result of a connectivity check."""
    print(f'The status of "{url}" is:', end=" ")
    if result:
        print('"Online!" 👍')
    else:
        print(f'"Offline?" 👎 \n  Error: "{error}"')


async def _asynchronous_check(urls,timeout):
    async def _check(url,timeout):
        error = ""
        try:
            result = await site_is_online_async(url,timeout)
        except Exception as e:
            result = False
            error = str(e)
        display_check_result(result, url, error)

    await asyncio.gather(*(_check(url,timeout) for url in urls))

def check(urls,timeout=5):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  # 'RuntimeError: There is no current event loop...'
        loop = None

    if loop and loop.is_running():
        print('Async event loop already running. Adding coroutine to the event loop.')
        tsk = loop.create_task(_asynchronous_check(urls,timeout))
        # ^-- https://docs.python.org/3/library/asyncio-task.html#task-object
        # Optionally, a callback function can be executed when the coroutine completes
        tsk.add_done_callback(
            lambda t: print(f'Task done with result={t.result()}  << return val of main()'))
    else:
        print('Starting new event loop')
        result = asyncio.run(_asynchronous_check(urls,timeout))	


In [8]:
import time
time.sleep(1)
check(timeout=3,urls=[])

Async event loop already running. Adding coroutine to the event loop.
Task done with result=None  << return val of main()


In [14]:
check(timeout=3, urls=["ssh://d01dc-ctrl303p.main.russianpost.ru","ssh://d01dc-ctrl504p.main.russianpost.ru","ssh://d01dc-ctrl404p.main.russianpost.ru:22","ssh://d01dc-ctrl404p.main.russianpost.ru:2222"])

Async event loop already running. Adding coroutine to the event loop.
The status of "ssh://d01dc-ctrl504p.main.russianpost.ru" is: "Offline?" 👎 
  Error: "[Errno -2] Name or service not known"
The status of "ssh://d01dc-ctrl303p.main.russianpost.ru" is: "Offline?" 👎 
  Error: "Permission denied"
The status of "ssh://d01dc-ctrl404p.main.russianpost.ru:22" is: "Offline?" 👎 
  Error: "Permission denied"
The status of "ssh://d01dc-ctrl404p.main.russianpost.ru:2222" is: "Offline?" 👎 
  Error: "timed out"
Task done with result=None  << return val of main()


In [15]:
check(timeout=3,urls=["https://gitlab.tools.russianpost.ru","https://jira.russianpost.ru","https://blablabla.russianpost.ru"])

Async event loop already running. Adding coroutine to the event loop.
The status of "https://blablabla.russianpost.ru" is: "Offline?" 👎 
  Error: "Cannot connect to host blablabla.russianpost.ru:443 ssl:False [Name or service not known]"
The status of "https://gitlab.tools.russianpost.ru" is: "Online!" 👍
The status of "https://jira.russianpost.ru" is: "Online!" 👍
Task done with result=None  << return val of main()


In [17]:
check(timeout=3,urls=
[
"http://d01dc-ctrl121.main.russianpost.ru:8088/cluster/scheduler",
"ssh://d01dc-ctrl303p.main.russianpost.ru",
"http://10.233.10.101:9102",
"https://gitlab.tools.russianpost.ru/",
"https://dc-ambari-clsu.russianpost.ru",
"http://datacloud-grafana.pochta.ru",
"https://dc-hue-str.russianpost.ru",
"https://dc-ambari-stru.russianpost.ru",
"https://dc-hue.russianpost.ru",
"http://int.reports.pochta.ru/dc-rest-test/api/test",
"https://uat-reports.pochta.ru",
"https://dc-hue-uat.russianpost.ru",
"http://kfm-uat.datacloud.russianpost.ru",
"https://dc-hue-dump.russianpost.ru",
"https://dc-yarn-stru.russianpost.ru",
"https://dc-yarn-clsu.russianpost.ru",
"https://dc-hue-clsp.russianpost.ru/",
"https://dc-hue-strp.russianpost.ru/",
"https://dc-ambari-clsp.russianpost.ru/#/login",
"https://dc-ambari-strp.russianpost.ru/#/login"
]
)

Async event loop already running. Adding coroutine to the event loop.
The status of "http://10.233.10.101:9102" is: "Online!" 👍
The status of "http://int.reports.pochta.ru/dc-rest-test/api/test" is: "Online!" 👍
The status of "https://dc-ambari-clsu.russianpost.ru" is: "Online!" 👍
The status of "https://gitlab.tools.russianpost.ru/" is: "Online!" 👍
The status of "http://d01dc-ctrl121.main.russianpost.ru:8088/cluster/scheduler" is: "Online!" 👍
The status of "https://uat-reports.pochta.ru" is: "Online!" 👍
The status of "https://dc-ambari-stru.russianpost.ru" is: "Online!" 👍
The status of "https://dc-hue-uat.russianpost.ru" is: "Online!" 👍
The status of "https://dc-hue-str.russianpost.ru" is: "Online!" 👍
The status of "https://dc-yarn-clsu.russianpost.ru" is: "Online!" 👍
The status of "https://dc-ambari-clsp.russianpost.ru/#/login" is: "Online!" 👍
The status of "http://datacloud-grafana.pochta.ru" is: "Offline?" 👎 
  Error: "Cannot connect to host datacloud-grafana.pochta.ru:80 ssl:False [